#### Prediction

In [6]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pandas as pd
import pickle

In [7]:
# load the trained model
model = load_model('model.keras')

f:\Note\M.Tech\DL\Code\Bank_Churn_using_ANN\Deep\Lib\site-packages\keras\src\saving\saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 8 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [8]:
# load the encoders and Scalers
with open('label_encoder_gender.pkl', 'rb') as file:
    label_encode_gender = pickle.load(file)
    
with open('onehot_encoder_geo.pkl', 'rb') as file:
    onehot_encoder_geo = pickle.load(file)
    
with open('scaler.pkl', 'rb') as file:
    scalers = pickle.load(file)

In [50]:
# input data
input_data = {
    'CreditScore' : 600,
    'Gender' : 'Male',
    'Geography' : 'France',
    'Age'  : 40,
    'Tenure' : 6,
    'Balance' : 60000,
    'NumOfProducts' : 2,
    'HasCrCard' : 1,
    'IsActiveMember' : 1,
    'EstimatedSalary' : 50000
}

In [53]:
geo_encoded = onehot_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded

f:\Note\M.Tech\DL\Code\Bank_Churn_using_ANN\Deep\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


array([[1., 0., 0.]])

In [54]:
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [57]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Gender,Geography,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,Male,France,40,6,60000,2,1,1,50000


In [58]:
input_df['Gender'] = label_encode_gender.transform(input_df['Gender'])

In [59]:
input_df = pd.concat([input_df.drop('Geography', axis=1), geo_encoded_df], axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,6,60000,2,1,1,50000,1.0,0.0,0.0


In [60]:
# Sca;ing up the input data
input_scaled = scalers.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359,  0.34567966, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [61]:
# Prediction Churn 
prediction = model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step


array([[0.04374706]], dtype=float32)

In [63]:
prediction_prob = prediction[0][0]
prediction_prob

np.float32(0.04374706)

In [64]:
if(prediction_prob >  0.5): print('Customer is likely to Churn')
else : print('Customer is not likely to churn')

Customer is not likely to churn
